In [ ]:
# type: ignore
from __future__ import annotations
from pprint import pprint
import cProfile
from typing import Any

import aiohttp

from vindicator import (
    DatabaseQuery,
    WynnApi,
    WynnDataRepository
)

In [ ]:
def className(obj: object) -> str:
    return obj.__class__.__name__

def getIndent(depth: int) -> str:
    return '    ' * depth

def printDictTypes(dict_: dict[str, Any], depth: int = 0) -> None:
    for key, value in dict_.items():
        if isinstance(value, dict):
            print(f'{getIndent(depth)}{key}: dict')
            printDictTypes(value, depth + 1)
        else:
            print(f'{getIndent(depth)}{key}: {className(value)}')

async def get_full_player_stats(username_or_uuid: str) -> tuple[dict[str, Any], dict[str, Any]]:
    async with aiohttp.ClientSession() as session:
        async with session.get(f"https://api.wynncraft.com/v3/player/{username_or_uuid}?fullResult") as resp:
            return (await resp.json(), dict(resp.headers))

async def get_guild_stats(guildname_or_prefix: str, is_prefix: bool = False) -> tuple[dict[str, Any], dict[str, Any]]:
    async with aiohttp.ClientSession() as session:
        async with session.get(f"https://api.wynncraft.com/v3/guild/{'prefix/' if is_prefix else ''}{guildname_or_prefix}") as resp:
            return (await resp.json(), dict(resp.headers))

async def get_online_players() -> tuple[dict[str, Any], dict[str, Any]]:
    async with aiohttp.ClientSession() as session:
        async with session.get("https://api.wynncraft.com/v3/player?identifier=uuid") as resp:
            return (await resp.json(), dict(resp.headers))

# Sandbox

In [15]:
from vindicator.api.wynn.player_response import PlayerResponse
from vindicator.db.wynndata.model.character_history.character_history import CharacterHistory
from vindicator.db.wynndata.model.character_info.character_info import CharacterInfo
from vindicator.db.wynndata.model.player_history.player_history import PlayerHistory
from vindicator.db.wynndata.model.player_info.player_info import PlayerInfo
from tests.mock_wynnapi import MockWynnApi

mockwynnapi = MockWynnApi()
data: list[PlayerResponse] = mockwynnapi.onlineplayerstats  # type: ignore

def withloop():
    playerinfo = []
    characterinfo = []
    playerhistory = []
    characterhistory = []
    for resp in data:
        playerinfo.extend(PlayerInfo.from_responses((resp,)))
        characterinfo.extend(CharacterInfo.from_responses((resp,)))
        playerhistory.extend(PlayerHistory.from_responses((resp,)))
        characterhistory.extend(CharacterHistory.from_responses((resp,)))

def without():
    playerinfo: tuple[PlayerInfo, ...] = PlayerInfo.from_responses(data)
    characterinfo: tuple[CharacterInfo, ...] = CharacterInfo.from_responses(data)
    playerhistory: tuple[PlayerHistory, ...] = PlayerHistory.from_responses(data)
    characterhistory: tuple[CharacterHistory, ...] = CharacterHistory.from_responses(data)

cProfile.run('withloop()', sort='time')
cProfile.run('without()', sort='time')

         542890 function calls in 0.362 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     3582    0.097    0.000    0.127    0.000 _strptime.py:309(_strptime)
     3582    0.077    0.000    0.274    0.000 character_history.py:75(<genexpr>)
    10089    0.023    0.000    0.030    0.000 uuid.py:139(__init__)
    35100    0.018    0.000    0.025    0.000 player.py:249(to_float)
     3582    0.010    0.000    0.041    0.000 character_info.py:21(<genexpr>)
    10089    0.009    0.000    0.045    0.000 uuid_field.py:9(to_bytes)
     3582    0.008    0.000    0.152    0.000 headers.py:25(get_datetime)
     3582    0.007    0.000    0.133    0.000 _strptime.py:565(_strptime_datetime)
     1314    0.005    0.000    0.029    0.000 player_history.py:37(<genexpr>)
     3582    0.005    0.000    0.142    0.000 date_field.py:13(to_datetime)
     3582    0.005    0.000    0.005    0.000 {built-in method _locale.setlocale}
    30267    

# Archive

In [ ]:
# += vs extend()
r = range(50_000_000)
def add():
    l = []
    l += tuple(r)
def extend():
    l = []
    l.extend(r)

cProfile.run('add()')
cProfile.run('extend()')

In [ ]:
# moto-bot repository structure
# from __future__ import annotations
# ID = TypeVar('ID', contravariant=True)
# T = TypeVar('T')
# S = TypeVar('S')
# U = TypeVar('U', bound=type)

# def interface(t: U) -> U: return t
# def abstract(t: U) -> U: return t

# @interface
# class Table(Generic[T, ID], Protocol):
#     def create(self, entity: T) -> bool: ...
#     def exists(self, id_: ID) -> bool: ...
#     def count(self) -> float: ...
#     def findOne(self, id_: ID) -> None | T: ...
#     def findAll(self) -> None | list[T]: ...
#     def update(self, entity: T) -> bool: ...
#     def delete(self, id_: ID) -> bool: ...

# @abstract
# class MariaRepository(ABC, Generic[S]):
#     def __init__(self, db: Any, logger: Any) -> None:
#         self._db: Any = db
#         self._logger: Any = logger
#     def _prepareStatement(self, connection: Any, sql: str, objects: list[object]) -> Any: ...
#     def _execute(self, sql: str, strings: list[object]) -> bool: ...
#     # def _execute(self, connection: Any, sql: str, objects: list[object]) -> bool: ...
#     def _executeQuery(self, sql: str, objects: list[object]) -> None | Any: ...
#     def _logResponseException(self, e: Any) -> None: ...
#     def _bindAll(self, res: Any) -> list[S]: ...
#     @abstractmethod
#     def _bind(self, res: Any) -> S: ...

# @interface
# class GuildId(Protocol):
#     def getName(self) -> str: ...

# class Guild(GuildId):
#     def __init__(self, name: str, prefix: str, createdAt: dt) -> None:
#         self._name: str = name
#         self._prefix: str = prefix
#         self._createdAt: dt = createdAt
#     @override
#     def getName(self) -> str: return self._name
#     def getPrefix(self) -> str: return self._prefix
#     def getCreatedAt(self) -> dt: return self._createdAt

# @interface
# class CharacterHistoryBase(Table[Guild, GuildId], Protocol):
#     def findAllIn(self, guildNames: list[str]) -> None | list[Guild]: ...
#     def findAllCaseInsensitive(self, guildName: str) -> None | list[Guild]: ...
#     def findAllByPrefix(self, prefix: str) -> None | list[Guild]: ...
#     def findAllByPrefixCaseInsensitive(self, prefix: str) -> None | list[Guild]: ...

# class CharacterHistoryTable(MariaRepository[Guild], CharacterHistoryBase):
#     """class::MariaGuildRepository"""
#     _DB_FORMAT: Any
#     def __init__(self, db: Any, logger: Any) -> None:
#         super().__init__(db, logger)
#     @override
#     def _bind(self, res: Any) -> Guild: ...
#     @override
#     def create(self, entity: Guild) -> bool: ...
#     @override
#     def exists(self, id_: GuildId) -> bool: ...
#     @override
#     def count(self) -> float: ...
#     @override
#     def findOne(self, id_: GuildId) -> None | Guild: ...
#     @override
#     def findAllIn(self, guildNames: list[str]) -> None | list[Guild]: ...
#     def findAllCaseInsensitive(self, guildName: str) -> None | list[Guild]: ...
#     def findAllByPrefix(self, prefix: str) -> None | list[Guild]: ...
#     def findAllByPrefixCaseInsensitive(self, prefix: str) -> None | list[Guild]: ...
#     @override
#     def findAll(self) -> None | list[Guild]: ...
#     @override
#     def update(self, entity: Guild) -> bool: ...
#     @override
#     def delete(self, id_: GuildId) -> bool: ...
